In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
from tensorflow.keras.models import load_model

In [2]:
import pandas as pd
df = pd.read_csv("latestdataset.csv")
df.drop(columns=["rain (mm)", "precipitation (mm)", "soil_moisture_0_to_7cm (m³/m³)"], inplace=True)
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df = df.astype(float)

In [13]:
# ---------- Load Scaler ----------
from sklearn.preprocessing import MinMaxScaler
scaler = joblib.load("scaler.pkl")  
df_scaled = pd.DataFrame(scaler.transform(df), index=df.index, columns=df.columns)

C:\Users\krati\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


In [14]:
# Create sequences

lookback = 48
horizon = 6
n_features = df_scaled.shape[1]

def create_sequences(data, lookback=48, horizon=6):
    X, y = [], []
    for i in range(len(data) - lookback - horizon + 1):
        X.append(data[i:i+lookback])
        y.append(data[i+lookback:i+lookback+horizon])
    return np.array(X), np.array(y)

X, y = create_sequences(df_scaled.values, lookback, horizon)

In [15]:
# train test val split

n = len(X)
train_idx = int(0.7 * n)
val_idx   = int(0.85 * n)

X_train, y_train = X[:train_idx], y[:train_idx]
X_val, y_val     = X[train_idx:val_idx], y[train_idx:val_idx]
X_test, y_test   = X[val_idx:], y[val_idx:]

In [16]:
# Flatten for XGBoost
X_val_flat  = X_val.reshape((X_val.shape[0], -1))
X_test_flat = X_test.reshape((X_test.shape[0], -1))
y_val_flat  = y_val.reshape((y_val.shape[0], -1))
y_test_flat = y_test.reshape((y_test.shape[0], -1))

# -------------------------------
# 1. Load models
# -------------------------------

In [17]:
lstm_model = load_model("best_lstm.keras")
gru_model  = load_model("best_gru.keras")
xgb_model  = joblib.load("xgb_multi_model.pkl")

# -------------------------------
# 2. Prepare validation data
# -------------------------------

In [18]:
n_features = X_val.shape[2]
horizon = y_val.shape[1]

X_val_flat = X_val.reshape((X_val.shape[0], -1))

# -------------------------------
# 3. Base model predictions on validation set
# -------------------------------

In [19]:
X_val_fixed = X_val[:, -48:, :]
X_test_fixed = X_test[:, -48:, :]
y_val_lstm = lstm_model.predict(X_val_fixed).reshape(X_val_fixed.shape[0], -1)
y_val_gru  = gru_model.predict(X_val_fixed).reshape(X_val_fixed.shape[0], -1)
y_val_xgb  = xgb_model.predict(X_val_flat)

# Stack base model predictions
X_meta = np.concatenate([y_val_lstm, y_val_gru, y_val_xgb], axis=1)
y_meta = y_val.reshape(X_val.shape[0], -1)  # flatten target for meta-model

411/411 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step
411/411 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step


In [22]:
print("y_val_lstm.shape:", y_val_lstm.shape)
print("y_val_gru.shape:", y_val_gru.shape)
print("y_val_xgb.shape:", y_val_xgb.shape)
print("X_meta.shape (to be fed to meta_model):", X_meta.shape)
print("Meta-model trained input size:", getattr(meta_model, "coef_", np.zeros((1,))).shape)


y_val_lstm.shape: (13146, 24)
y_val_gru.shape: (13146, 24)
y_val_xgb.shape: (13146, 24)
X_meta.shape (to be fed to meta_model): (13146, 72)
Meta-model trained input size: (24, 72)


# -------------------------------
# 4. Train meta-model
# -------------------------------

In [23]:
X_meta = np.concatenate([y_val_lstm, y_val_gru, y_val_xgb], axis=1)
y_meta = y_val.reshape(X_val.shape[0], -1)  # same as before

meta_model = Ridge(alpha=1.0)
meta_model.fit(X_meta, y_meta)
joblib.dump(meta_model, 'meta_model.pkl')  # Save new meta_model


['meta_model.pkl']

# -------------------------------
# 5. Prepare test set predictions
# -------------------------------

In [24]:
X_test_flat = X_test.reshape((X_test.shape[0], -1))

y_test_lstm = lstm_model.predict(X_test_fixed).reshape(X_test_fixed.shape[0], -1)
y_test_gru  = gru_model.predict(X_test_fixed).reshape(X_test_fixed.shape[0], -1)
y_test_xgb  = xgb_model.predict(X_test_flat)

X_test_meta = np.concatenate([y_test_lstm, y_test_gru, y_test_xgb], axis=1)

# Final stacked predictions
y_pred_stacked = meta_model.predict(X_test_meta)
y_pred_stacked = y_pred_stacked.reshape((X_test.shape[0], horizon, n_features))

411/411 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
411/411 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step


# -------------------------------
# 6. Evaluate stacked predictions
# -------------------------------

In [25]:
features = df.columns.tolist()

results = {}
avg_results = {}

for f_idx, feature in enumerate(features):
    results[feature] = {}
    mae_list, rmse_list, r2_list = [], [], []
    for h in range(horizon):
        yt = y_test[:, h, f_idx]
        yp = y_pred_stacked[:, h, f_idx]

        mae  = mean_absolute_error(yt, yp)
        rmse = np.sqrt(mean_squared_error(yt, yp))
        r2   = r2_score(yt, yp)

        results[feature][f"Horizon_{h+1}"] = {"MAE": mae, "RMSE": rmse, "R2": r2}

        mae_list.append(mae)
        rmse_list.append(rmse)
        r2_list.append(r2)

    avg_results[feature] = {
        "MAE": np.mean(mae_list),
        "RMSE": np.mean(rmse_list),
        "R2": np.mean(r2_list)
    }

# -------------------------------
# 7. Show results in tabular form
# -------------------------------

In [26]:
# Per feature & horizon
rows = []
for feat, horizons in results.items():
    for h, metrics in horizons.items():
        row = {"Feature": feat, "Horizon": h}
        row.update(metrics)
        rows.append(row)

results_df = pd.DataFrame(rows)
print("\n===== Stacking Ensemble Metrics per Feature & Horizon =====")
print(results_df)

# Average per feature
avg_df = pd.DataFrame(avg_results).T.reset_index().rename(columns={"index":"Feature"})
print("\n===== Stacking Ensemble Average Metrics per Feature =====")
print(avg_df)


===== Stacking Ensemble Metrics per Feature & Horizon =====
                           Feature    Horizon       MAE      RMSE        R2
0              temperature_2m (°C)  Horizon_1  0.013808  0.021290  0.979503
1              temperature_2m (°C)  Horizon_2  0.018646  0.026886  0.967319
2              temperature_2m (°C)  Horizon_3  0.021508  0.030355  0.958348
3              temperature_2m (°C)  Horizon_4  0.023126  0.032317  0.952802
4              temperature_2m (°C)  Horizon_5  0.024323  0.033723  0.948620
5              temperature_2m (°C)  Horizon_6  0.025398  0.034917  0.944935
6         relative_humidity_2m (%)  Horizon_1  0.023691  0.036299  0.975029
7         relative_humidity_2m (%)  Horizon_2  0.033845  0.048199  0.955973
8         relative_humidity_2m (%)  Horizon_3  0.040074  0.055884  0.940817
9         relative_humidity_2m (%)  Horizon_4  0.044133  0.060882  0.929761
10        relative_humidity_2m (%)  Horizon_5  0.047012  0.064515  0.921139
11        relative_humidity

# -------------------------------
# 8. Save stacked model for later use
# -------------------------------

In [45]:
"""joblib.dump({
    "meta_model": meta_model,
    "lstm_model": lstm_model,
    "gru_model": gru_model,
    "xgb_model": xgb_model
}, "stacked_ensemble.pkl")
print("✅ Stacked ensemble saved as 'stacked_ensemble.pkl'")"""

'joblib.dump({\n    "meta_model": meta_model,\n    "lstm_model": lstm_model,\n    "gru_model": gru_model,\n    "xgb_model": xgb_model\n}, "stacked_ensemble.pkl")\nprint("✅ Stacked ensemble saved as \'stacked_ensemble.pkl\'")'

In [27]:
# Save only meta-model + XGBoost, not Keras models
"""joblib.dump({
    "meta_model": meta_model,
    "xgb_model": xgb_model
}, "stacked_ensemble.pkl")
print("✅ Stacked ensemble saved (without pickling Keras models)")"""


'joblib.dump({\n    "meta_model": meta_model,\n    "xgb_model": xgb_model\n}, "stacked_ensemble.pkl")\nprint("✅ Stacked ensemble saved (without pickling Keras models)")'

In [28]:
import joblib

# meta_model and xgb_model should already be defined from your training cell
pipeline = {
    "meta_model": meta_model,     # sklearn Ridge
    "xgb_model": xgb_model,       # joblib XGBoost model
    "scaler": scaler,             # MinMaxScaler used for training
    "features": df.columns.tolist(),  # feature order
    "look_back": lookback,
    "horizon": horizon
}

joblib.dump(pipeline, "stacked_pipeline.pkl")
print("✅ stacked_pipeline.pkl saved (contains meta_model, xgb, scaler and metadata)")
# Ensure Keras models are saved (you already have these lines in your notebook)
lstm_model.save("best_lstm.keras")
gru_model.save("best_gru.keras")
print("✅ Keras models saved: best_lstm.keras, best_gru.keras")


✅ stacked_pipeline.pkl saved (contains meta_model, xgb, scaler and metadata)
✅ Keras models saved: best_lstm.keras, best_gru.keras


In [17]:
saved = joblib.load("stacked_ensemble.pkl")
meta_model = saved["meta_model"]
xgb_model  = saved["xgb_model"]

# Now you can do:
# y_pred_xgb = xgb_model.predict(X_input_flat)
# y_pred_stacked = meta_model.predict(X_meta_input)


In [18]:
with open('stacked_ensemble.pkl', 'rb') as f:
    model_dict = pickle.load(f)

# Extract the trained ensemble model
model = model_dict['meta_model']   # or whatever key you used

# Then use it normally
y_pred = model.predict(X_input).reshape(horizon, len(FEATURES))


NameError: name 'pickle' is not defined

In [29]:
print(lstm_model.input_shape)  # Should print (None, 48, 4)
print("GRU expected input shape:", gru_model.input_shape) 
print("X_val shape:", X_val.shape)

(None, 48, 4)
GRU expected input shape: (None, 48, 4)
X_val shape: (13146, 48, 4)
